(Graph(num_nodes=15, num_edges=45,
      ndata_schemes={}
      edata_schemes={}), tensor(0))


In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1, 2], [0, 3], [1, 5]], dtype=torch.float)

# x represents nodes and node features
# edges represent the edges between nodes
data = Data(x=x, edge_index=edge_index)
print(data)
# Data(edge_index=[2, 4], x=[3, 1])

Data(x=[3, 2], edge_index=[2, 4])


In [3]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

dataset[0]

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [8]:
# Test on how to format agents to use data
batch_size = 5
seq_len = 3
num_agents = 4
features = 2

test = torch.randn((batch_size, seq_len, num_agents, features))
permuted = test.permute(0, 2, 1, 3).contiguous() # (batch_size, num_agents, seq_len, features)
lstm_input = permuted.view(batch_size * num_agents, seq_len, features)

print(lstm_input.shape)

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.data.batch import Batch
from torch_geometric.nn import GCNConv
# # Test on converting from lstm output to graph input
# batch_size = 64
# hidden_dim = 8
# num_agents = 60
# lstm_output = torch.randn((batch_size * num_agents, hidden_dim)).requires_grad_()

batch_size = 64
seq_len = 5
num_agents = 60
features = 2
hidden_dim = 8

input_tensor = torch.randn((batch_size, seq_len, num_agents, features))
permuted = input_tensor.permute(0, 2, 1, 3).contiguous() # (batch_size, num_agents, seq_len, features)
lstm_input = permuted.view(batch_size * num_agents, seq_len, features)

lstm_layer = nn.LSTM(input_size=features, 
                hidden_size=hidden_dim, 
                num_layers=1, 
                batch_first=True)

out, (hn, cn) = lstm_layer(lstm_input)
print(hn.shape)

reshaped = hn.view(batch_size, num_agents, hidden_dim)
print(reshaped.shape)

data_list = []
for i in range(batch_size):
    x = reshaped[i]
    edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

    data = Data(x=x, edge_index=edge_index)
    data_list.append(data)
    # print(data.x.requires_grad)
# print(reshaped.shape)

batch = Batch()
batched_data = batch.from_data_list(data_list)

conv1 = GCNConv(features, 16)
print(conv1(batched_data).shape)

# print(batched_data)
# batched_data[0]


: 